In [0]:
%run ../../config/config 

In [0]:
%run ../../config/sqlconfig

In [0]:
from pyspark.sql.types import StructType, StringType, IntegerType, StructField, DecimalType
from pyspark.sql.functions import col, lit,filter
import os

In [0]:
class SalesSilverStream:
    def __init__(self, spark):
        self.spark = spark

    def run(self):
        df = (
            self.spark.readStream
            .table("workspace.etl_practice.fact_sales")
        )

        # Remove duplicates
        # Use appropriate business keys for deduplication
        dedup_df = df.dropDuplicates()

        (
            dedup_df.writeStream
            .trigger(availableNow=True)
            .format("delta")
            .outputMode("append")
            .option("checkpointLocation", f"{base_path}/_checkpoints/sales_silver")
            .table("workspace.etl_practice.silver_fact_sales")
        )

        print("Running Sales Silver Stream")

In [0]:
obj = SalesSilverStream(spark)
obj.run()